In [1]:
import cv2
from tensorflow.keras.models import load_model 
import numpy as np  
from pygame import mixer 
import os 
import face_recognition 
import datetime 
import csv 
import time 
import sqlite3
import smtplib
import dlib
from imutils import face_utils

2023-07-04 11:45:22.819773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 11:45:23.067425: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-07-04 11:45:23.067464: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-04 11:45:23.123391: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already be

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
cd "/home/user/Downloads/Project_new"    


/home/user/Downloads/Project_new


In [3]:

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


model = load_model('model/model_eyes1.h5')
model2 = load_model('model/model_yawn1.h5')

2023-07-04 11:45:31.766806: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-04 11:45:31.766965: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (user-HP-Laptop-15-bs0xx): /proc/driver/nvidia/version does not exist
2023-07-04 11:45:31.768057: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:

mixer.init()
sound= mixer.Sound(r'/home/user/Downloads/Project_new/alarm.wav')
cap = cv2.VideoCapture(0)

eye_score = 0
eye_thicc=2
yawn_score = 0
yawn_thicc=2
eye_counter=0
yawn_counter=0
font = cv2.FONT_HERSHEY_COMPLEX_SMALL


# Load images and labels for face recognition
known_face_encodings = []
known_face_labels = []


img_path_1 = "/home/user/Downloads/Project_new/Training_images/Ajnas.jpg"
img_path_2 = "/home/user/Downloads/Project_new/Training_images/Aswanth_A.jpg"

img_1 = cv2.imread(img_path_1)
img_2 = cv2.imread(img_path_2)


label_1 = "Ajnas"
label_2 = "Aswanth A"


known_face_encodings.append(face_recognition.face_encodings(img_1)[0])
known_face_encodings.append(face_recognition.face_encodings(img_2)[0])

known_face_labels = [label_1, label_2]

# Create CSV file and headers
with open('data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Eye Status', 'Yawn Status', 'Time'])



def send_email():
    sender_email = "touristbusdriver1@gmail.com"
    receiver_email = "touristbusadmi@gmail.com"
    password = "grww etbe bonf lpox"

    subject = "Alert!"
    message = "The alert sound is continuously playing the driver getting drowsy."

    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, f"Subject: {subject}\n\n{message}")
        print("Email sent successfully")
    except Exception as e:
        print(f"Error: {str(e)}")
    finally:
        server.quit()



while True:
        ret, frame = cap.read()
        height,width = frame.shape[0:2]

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    #COLOR_BGR2GRAY
        faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
        eyes= eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)
        
        cv2.rectangle(frame, (0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)


        
         # Detect faces in the frame
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)


        # Loop over the detected faces and draw a rectangle around them
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Check if the detected face matches with any of the known faces
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_labels[first_match_index]

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, name, (left, top - 6), font, 0.5, (0, 0, 255), 2)
       

        
        def calculate_mouth_aspect_ratio(shape):
                left_mouth = shape[48:60]  # Indices of the left mouth landmarks
                right_mouth = shape[60:68]  # Indices of the right mouth landmarks

                # Calculate the distances between the mouth landmarks
                left_distance = np.mean(np.abs(left_mouth[1:] - left_mouth[:-1]))
                right_distance = np.mean(np.abs(right_mouth[1:] - right_mouth[:-1]))

                # Calculate the mouth aspect ratio
                mouth_aspect_ratio = (left_distance + right_distance) / (2 * mouth_width)


                return mouth_aspect_ratio


        for (x,y,w,h) in faces:
                cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color= (255,0,0),thickness=3 )
                mouth= frame[y:y+h, x:x+w]
                mouth= cv2.resize(mouth,(80, 80))
                mouth= mouth/255
                mouth=mouth.reshape(80, 80,3)
                mouth= np.expand_dims(mouth,axis=0)
                # preprocessing is done now model prediction
                yawn_prediction = model2.predict(mouth)
                print([np.round(x*100, 2) for x in yawn_prediction])
                # print(prediction)
                if yawn_prediction[0][0]<yawn_prediction[0][1]:
                        yawn_score=yawn_score+1
                        print("Yawn")
                        yawn_status = "Yawn"
                        cv2.putText(frame,'Yawn',(10,height-380),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,0,255), thickness=1,lineType=cv2.LINE_AA)
                else:
                        yawn_score=yawn_score-1
                        print("No Yawn")
                        yawn_status = "No_Yawn"
                        cv2.putText(frame,'No Yawn',(10,height-380),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(0,0,255),thickness=1,lineType=cv2.LINE_AA)


        for (ex,ey,ew,eh) in eyes:  
                cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color=(255,0,0), thickness=3 )
                # preprocessing steps
                eye= frame[ey:ey+eh,ex:ex+ew]
                eye= cv2.resize(eye,(80,80))
                eye= eye/255
                eye= eye.reshape(80,80,3)
                eye= np.expand_dims(eye,axis=0)
                # preprocessing is done now model prediction
                eye_prediction = model.predict(eye)
                #print(eye_prediction)
                # if eyes are closed
                if eye_prediction[0][0]>0.8:
                        eye_score=eye_score+1
                        eye_status = "Closed"
                        cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255 ),thickness=1,lineType=cv2.LINE_AA)
                # if eyes are open
                else:
                        eye_score=eye_score-1
                        eye_status = "Open"
                        cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),thickness=1,lineType=cv2.LINE_AA)  

        if(yawn_score<0):
                        yawn_score=0
        cv2.putText(frame,'Score:'+str(yawn_score),(150,height-380), font,1,(0,0,255),1,cv2.LINE_AA)
        cv2.putText(frame,'Counter_y:'+str(int(yawn_counter/5)),(400,height-380), font, 1,(0,0,255),1,cv2.LINE_AA)
        if(yawn_score>7):
                #person is feeling sleepy so we beep the alarm
                if(yawn_counter/7>=3):
                        yawn_counter=0
                       
                try:
                        sound.play()
                        send_email()
                        yawn_counter+=1
                except: # isplaying = False
                        pass
                if(yawn_thicc<8):
                        yawn_thicc= yawn_thicc+2
                else:
                        yawn_thicc=yawn_thicc-2
                        if(yawn_thicc<2):
                                yawn_thicc=2
                cv2.rectangle(frame,(0,0),(width,height),(0,0,255),yawn_thicc)

        for face in faces:
                        shape = predictor(gray, face)
                        shape = face_utils.shape_to_np(shape)

                        mouth_landmarks = shape[48:68]  # Extract the mouth landmarks (indices 48-67)
                        mouth_aspect_ratio = calculate_mouth_aspect_ratio(mouth_landmarks)

        

        if(eye_score<0):
                        eye_score=0
        cv2.putText(frame,'Score:'+str(eye_score),(100,height-20), font,1,(255,255,255),1,cv2.LINE_AA)
        cv2.putText(frame,'Counter_e:'+str(int(eye_counter/9)),(400,height-20), font, 1,(0,0,255),1,cv2.LINE_AA)
        if(eye_score>15):
                #person is feeling sleepy so we beep the alarm
                if(eye_counter/8>=3):
                        eye_counter=0
                       
                try:
                        sound.play()
                        time.sleep(10)
                        send_email()
                        eye_counter+=1

                except: # isplaying = False
                        pass
                if(eye_thicc<15):
                        eye_thicc= eye_thicc+2
                else:
                        eye_thicc=eye_thicc-2
                        if(eye_thicc<2):
                                eye_thicc=2
                                
                cv2.rectangle(frame,(0,0),(width,height),(0,0,255),eye_thicc)               
        cv2.imshow('frame',frame)
        if cv2.waitKey(33) & 0xFF==ord('q'):
                break

         # Get current time
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Save data in CSV file
        with open('data.csv', mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([name,eye_status, yawn_status, current_time])     

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 2s 2s/step
[array([36.65, 63.35], dtype=float32)]
Yawn
1/1 [==============================] - 0s 106ms/step


NameError: name 'predictor' is not defined